In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import requests
import datetime
import dateutil
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
import ast 
from collections import Counter

In [3]:
#language = "en"
#language = "it"
#language = "es"
language = "en"
codes = ['en', 'it', 'es', 'es']
years = list(range(2019, 2020 + 1))

In [3]:
df = pd.read_csv("events_2015-2020_en.csv", converters={'redirects': pd.eval, 'list_views_7_days':pd.eval, 'list_views_before':pd.eval, 'list_views_7_days_redirects':pd.eval, 'categories': pd.eval,})
df

,event_url,event_label,date,country,start_date,end_date,event_date,event_id,pagetitle,redirects,...,list_views_7_days_redirects,diff_days,list_views_before,categories,economic_region,continent,cat,list_edits_before,list_edits,list_edits_redirects
0,http://www.wikidata.org/entity/Q20647547,2015 Kazakhstan First Division,2015-01-01T00:00:00Z,Kazakhstan,2015-04-10T00:00:00Z,2015-11-14T00:00:00Z,2015-04-10T00:00:00Z,Q20647547,2015 Kazakhstan First Division,[],...,[],86,[],[Category:2014–15 in European second tier asso...,Global South,Asia,sports,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]",[]
1,http://www.wikidata.org/entity/Q20647776,2015 Greek football scandal,2015-01-01T00:00:00Z,Greece,2015-04-06T00:00:00Z,NaN,2015-04-06T00:00:00Z,Q20647776,2015 Greek football match-fixing scandal,"[2015 Greek football scandal, 2015 Greek footb...",...,[],35,[],"[Category:2014–15 in Greek football, Category:...",Global North,Europe,sports,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,http://www.wikidata.org/entity/Q20647949,2015 Washington wildfires,2015-01-01T00:00:00Z,United States,2015-06-28T00:00:00Z,NaN,2015-06-28T00:00:00Z,Q20647949,2015 Washington wildfires,"[2015 washington state wildfires, 2015 Washing...",...,"[130, 72, 61, 40]",1,[],"[Category:2015 in Washington (state), Category...",Global North,North America,disaster,"[0, 0, 0, 0, 0, 0]","[0, 13, 2, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,http://www.wikidata.org/entity/Q20648362,2015 Reinert Open,2015-01-01T00:00:00Z,Germany,2015-07-06T00:00:00Z,2015-07-12T00:00:00Z,2015-07-06T00:00:00Z,Q20648362,2015 Reinert Open,[],...,[],-6,"[9, 33, 23, 20, 43]","[Category:2015 ITF Women's Circuit, Category:2...",Global North,Europe,sports,"[0, 1, 0, 1, 0, 0]","[1, 1, 0, 0, 0, 4]",[]
4,http://www.wikidata.org/entity/Q18348667,2015 Cork Senior Hurling Championship,2015-01-01T00:00:00Z,Ireland,2015-04-11T00:00:00Z,NaN,2015-04-11T00:00:00Z,Q18348667,2015 Cork Senior Hurling Championship,[],...,[],-181,[],[Category:2015 senior hurling county champions...,Global North,Europe,sports,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 5, 0, 0]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10564,http://www.wikidata.org/entity/Q105616790,Results of the 2020 New Hampshire House of Rep...,2020-11-03T00:00:00Z,United States,NaN,NaN,2020-11-03T00:00:00Z,Q105616790,Results of the 2020 New Hampshire House of Rep...,[],...,[],95,[],"[Category:2020 New Hampshire elections, Catego...",Global North,North America,politics,"[0, 0, 0, 0, 0, 0]",[],[]
10565,http://www.wikidata.org/entity/Q100791760,protests against the tightening of the abortio...,2020-01-01T00:00:00Z,Poland,2020-10-22T00:00:00Z,http://www.wikidata.org/.well-known/genid/d805...,2020-10-22T00:00:00Z,Q100791760,2020–2021 women's strike protests in Poland,"[2020 Polish abortion protests, October 2020 P...",...,[5406],5,[],"[Category:2020 protests, Category:Abortion-rig...",Global North,Europe,NaN,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 21]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10566,http://www.wikidata.org/entity/Q85711929,2019–20 Confederation presidential primary,2020-01-18T00:00:00Z,United States,NaN,NaN,2020-01-18T00:00:00Z,Q85711929,2019–20 Confederation presidential primary,"[2020 Confederation presidential primary, 2019...",...,"[93, 36, 31, 28, 15, 18, 18]",-80,"[241, 209, 144, 87, 75, 96, 80]","[Category:2019 elections in Poland, Category:2...",Global North,North America,politics,"[4, 3, 3, 0, 1, 1]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10567,http://www.wikidata.org/entity/Q85735165,2020 Civic Democratic Party leadership election,2020-01-18T00:00:00Z,Czech Republic,NaN,NaN,2020-01-18T00:00:00Z,Q85735165,2020 Civic Democratic Party leadership election,[],...,[],-54,"[8, 4, 4, 4, 14, 4, 13]",[Category:2020 elections in the Czech Republic...,Global North,Europe,politics,"[0, 0, 0, 0, 0, 0]","[3, 0, 0, 0, 4, 0]",[]


In [ ]:
"""
 SELECT ?event ?eventLabel ?date ?countryLabel ?start ?end ?instanceofLabel ?partofLabel ?facetofLabel ?sportLabel ?deathsLabel ?injuredLabel
    WHERE
    {
      ?event  p:P585/ps:P585 ?date.
      OPTIONAL { ?event wdt:P580 ?start. }
      OPTIONAL { ?event wdt:P582 ?end. }
      .FILTER(?date <= "+2020-01-31T00:00:00Z"^^xsd:dateTime)
      .FILTER(?date >= "+2020-01-05T00:00:00Z"^^xsd:dateTime)
      ?event wdt:P17 ?country.
      OPTIONAL{?event wdt:P31 ?instanceof.}
      OPTIONAL{?event wdt:P361 ?partof.}
      OPTIONAL{?event wdt:P1269 ?facetof.}
      OPTIONAL{?event wdt:P641 ?sport.}
      OPTIONAL{?event wdt:P1120 ?deaths.}
      OPTIONAL{?event wdt:P1339 ?injured.}
      
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
"""

In [4]:
def queryEventData(year):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery("""
    SELECT ?event ?eventLabel ?date ?countryLabel ?start ?end
    WHERE
    {
      ?event  p:P585/ps:P585 ?date.
      OPTIONAL { ?event wdt:P580 ?start. }
      OPTIONAL { ?event wdt:P582 ?end. }""" + 
      f'.FILTER(?date <= "+{year}-12-31T00:00:00Z"^^xsd:dateTime)' +
      f'.FILTER(?date >= "+{year}-01-01T00:00:00Z"^^xsd:dateTime).' + 
      """
      ?event wdt:P17 ?country.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.io.json.json_normalize(results['results']['bindings'])
    df = results_df[['event.value', 'eventLabel.value', 'date.value', 'countryLabel.value', 'start.value', 'end.value']]
    df = df.rename(columns={'event.value': 'event_url', 
                            'eventLabel.value': 'event_label',
                            'date.value': 'date',
                            'countryLabel.value': 'country',
                            'start.value': 'start_date',
                            'end.value': 'end_date'})
    return df

def getDate(date, start_date):
    if pd.isnull(start_date):
        return date
    else:
        return start_date
    
def getEventID(event):
    return event.split("entity/")[-1]

In [ ]:
# query event data from dewiki from 2020
list_dfs = []
for year in years:
    df_year = queryEventData(year)
    # select event_date -- if start_date is set -> start_date else: point_in_time (date)
    df_year['event_date'] = df_year.apply(lambda row : getDate(row['date'],row['start_date']), axis = 1)
    # extract wikidata item id
    df_year['event_id'] = df_year.apply(lambda row : getEventID(row['event_url']), axis = 1)
    list_dfs.append(df_year)
dfs_event = pd.concat(list_dfs)
dfs_event.to_csv(r'dfs_event_wd.csv', index = False)

In [49]:
#################################################################################################################

In [105]:
def getPagetitle(event_id, language):
    wiki = language+'wiki'
    URL = "https://www.wikidata.org/w/api.php"
    PARAMS = {
        "action": "wbgetentities",
        "format": "json",
        "props":"sitelinks",
        "sitefilter": wiki,
        "ids": event_id
    }
    data = requests.get(url=URL, params=PARAMS).json()
    try:
        title = data['entities'][event_id]['sitelinks'][wiki]['title']
    except:
        title = ""
    return title

In [104]:
dfs_event['pagetitle'] = dfs_event.apply(lambda row : getPagetitle(row['event_id'], language), axis = 1)
# keep wikidata items with pagetitle 
#df = df[df['pagetitle'] != ""]
dfs_event.to_csv(r'dfs_event_wd_title.csv', index = False)

In [ ]:
#df2020 = df
#df2020.to_csv (r'df2020_de.csv', index = False, header=True)

In [55]:
df = df2015.append(df2016)
df = df.append(df2017)
df = df.append(df2018)
df = df.append(df2019)
df = df.append(df2020)
df

,event_url,event_label,date,country,start_date,end_date,event_date,event_id,pagetitle
14,http://www.wikidata.org/entity/Q19868605,2015 French Open – Women's Singles,2015-01-01T00:00:00Z,France,NaN,NaN,2015-01-01T00:00:00Z,Q19868605,French Open 2015/Dameneinzel
15,http://www.wikidata.org/entity/Q19868613,2015 French Open – Mixed Doubles,2015-01-01T00:00:00Z,France,NaN,NaN,2015-01-01T00:00:00Z,Q19868613,French Open 2015/Mixed
59,http://www.wikidata.org/entity/Q19883468,2015 Slovenia International Badminton Champion...,2015-01-01T00:00:00Z,Slovenia,2015-05-07T00:00:00Z,2015-05-10T00:00:00Z,2015-05-07T00:00:00Z,Q19883468,Slovenia International 2015
61,http://www.wikidata.org/entity/Q19885461,Q19885461,2015-01-01T00:00:00Z,Turkey,NaN,NaN,2015-01-01T00:00:00Z,Q19885461,TEB BNP Paribas İstanbul Open 2015/Qualifikation
66,http://www.wikidata.org/entity/Q19912329,2015 Lecoq Seoul Open,2015-01-01T00:00:00Z,South Korea,2015-05-11T00:00:00Z,2015-05-17T00:00:00Z,2015-05-11T00:00:00Z,Q19912329,Lecoq Seoul Open 2015
...,...,...,...,...,...,...,...,...,...
5543,http://www.wikidata.org/entity/Q100534658,Preis der Diana 2020,2020-08-02T00:00:00Z,Germany,NaN,NaN,2020-08-02T00:00:00Z,Q100534658,162. Preis der Diana 2020
5600,http://www.wikidata.org/entity/Q19863352,2020 United States Senate elections,2020-11-03T00:00:00Z,United States of America,NaN,NaN,2020-11-03T00:00:00Z,Q19863352,Wahl zum Senat der Vereinigten Staaten 2020
5601,http://www.wikidata.org/entity/Q22923830,2020 United States presidential election,2020-11-03T00:00:00Z,United States of America,NaN,NaN,2020-11-03T00:00:00Z,Q22923830,Präsidentschaftswahl in den Vereinigten Staate...
5603,http://www.wikidata.org/entity/Q48834828,2020 United States Senate election in Alabama,2020-11-03T00:00:00Z,United States of America,NaN,NaN,2020-11-03T00:00:00Z,Q48834828,Wahl zum Senat der Vereinigten Staaten in Alab...


In [ ]:
##################################################################################################################

In [4]:
def getRedirects(pagetitle, language):
    URL = "https://"+language+".wikipedia.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "titles": pagetitle,
        "prop": "redirects",
        "rdlimit":"100"
    }
    data = requests.get(url=URL, params=PARAMS).json()
    page = data["query"]["pages"]

    redirects = []
    try:
        for key, value in page.items():
            for redirect in value["redirects"]:
                redirects.append(redirect["title"])
    except:
        redirects = []
    return redirects

In [5]:
df['redirects'] = df.apply(lambda row : getRedirects(row['pagetitle'],language), axis = 1)
df.head()

,event_url,event_label,date,country,start_date,end_date,event_date,event_id,pagetitle,redirects
0,http://www.wikidata.org/entity/Q18348657,2015 Cannes Film Festival,2015-01-01T00:00:00Z,France,NaN,NaN,2015-01-01T00:00:00Z,Q18348657,Internationale Filmfestspiele von Cannes 2015,"[Internationales Filmfestival von Cannes 2015,..."
1,http://www.wikidata.org/entity/Q18413455,2015 UEC European Track Championships,2015-01-01T00:00:00Z,Switzerland,NaN,2015-10-18T00:00:00Z,2015-01-01T00:00:00Z,Q18413455,UEC-Bahn-Europameisterschaften 2015,[]
2,http://www.wikidata.org/entity/Q18413459,2015 US Open,2015-01-01T00:00:00Z,United States of America,2015-08-31T00:00:00Z,2015-09-13T00:00:00Z,2015-08-31T00:00:00Z,Q18413459,US Open 2015,[]
3,http://www.wikidata.org/entity/Q18450378,2015 European Short Course Swimming Championships,2015-01-01T00:00:00Z,Israel,2015-12-02T00:00:00Z,2015-12-06T00:00:00Z,2015-12-02T00:00:00Z,Q18450378,Kurzbahneuropameisterschaften 2015,[]
4,http://www.wikidata.org/entity/Q18507280,2015 World Ice Hockey Championships,2015-01-01T00:00:00Z,Czech Republic,NaN,NaN,2015-01-01T00:00:00Z,Q18507280,Eishockey-Weltmeisterschaft 2015,[]


In [7]:
##################################################################################################################

In [8]:
def getPageCreationDate(pagetitle, language):
    URL = "https://"+language+".wikipedia.org/w/api.php"
    PARAMS = {
        "action": "query",
        "prop": "revisions",
        "rvlimit":"1",
        "titles": pagetitle,
        "rvprop": "timestamp",
        "rvdir": "newer",
        'format': 'json',
    }

    data = requests.get(url=URL, params=PARAMS).json()
    try:
        page = next(iter(data["query"]["pages"].values()))
        pagecreation = page["revisions"][0]["timestamp"]
    except:
        pagecreation = ""
    return pagecreation

def calcDiffDays(event_date, page_creation):
    page_creation = datetime.strptime(page_creation,'%Y-%m-%dT%H:%M:%SZ')
    event_date = datetime.strptime(event_date,'%Y-%m-%dT%H:%M:%SZ')
    diff = page_creation - event_date
    return diff.days

In [ ]:
df = df[df.page_creation.isnull() == False]

In [9]:
df['page_creation'] = df.apply(lambda row : getPageCreationDate(row['pagetitle'],language), axis = 1)
# calc page creation date - event date 
# positiv if page was created after event happend
# negative if page was created before event happend
df['diff_days'] = df.apply(lambda row : calcDiffDays(row['event_date'],row['page_creation']), axis = 1)
df.head()

,event_url,event_label,date,country,start_date,end_date,event_date,event_id,pagetitle,redirects,page_creation,diff_days
0,http://www.wikidata.org/entity/Q18348657,2015 Cannes Film Festival,2015-01-01T00:00:00Z,France,NaN,NaN,2015-01-01T00:00:00Z,Q18348657,Internationale Filmfestspiele von Cannes 2015,"[Internationales Filmfestival von Cannes 2015,...",2014-10-05T18:36:54Z,-88
1,http://www.wikidata.org/entity/Q18413455,2015 UEC European Track Championships,2015-01-01T00:00:00Z,Switzerland,NaN,2015-10-18T00:00:00Z,2015-01-01T00:00:00Z,Q18413455,UEC-Bahn-Europameisterschaften 2015,[],2014-09-13T17:11:44Z,-110
2,http://www.wikidata.org/entity/Q18413459,2015 US Open,2015-01-01T00:00:00Z,United States of America,2015-08-31T00:00:00Z,2015-09-13T00:00:00Z,2015-08-31T00:00:00Z,Q18413459,US Open 2015,[],2014-10-07T16:23:12Z,-328
3,http://www.wikidata.org/entity/Q18450378,2015 European Short Course Swimming Championships,2015-01-01T00:00:00Z,Israel,2015-12-02T00:00:00Z,2015-12-06T00:00:00Z,2015-12-02T00:00:00Z,Q18450378,Kurzbahneuropameisterschaften 2015,[],2015-11-23T12:07:35Z,-9
4,http://www.wikidata.org/entity/Q18507280,2015 World Ice Hockey Championships,2015-01-01T00:00:00Z,Czech Republic,NaN,NaN,2015-01-01T00:00:00Z,Q18507280,Eishockey-Weltmeisterschaft 2015,[],2010-05-21T22:39:10Z,-1686


In [10]:
##################################################################################################################

In [8]:
headers = {"User-Agent": "steinkasserer@student.tugraz.at"}

def extractViews(json_data):
    list_views = []
    if "items" in json_data:
        for i in json_data["items"]:
            list_views.append(i["views"])
    return list_views
            
def getViews(pagetitle, event_date, days, language):   
    from_date = datetime.strptime(event_date,'%Y-%m-%dT%H:%M:%SZ')      
    time_range = dateutil.relativedelta.relativedelta(days=days)
    to_date = from_date + time_range
    
    from_date = from_date.strftime('%Y%m%d')
    to_date = to_date.strftime('%Y%m%d')
    
    url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/"+language+".wikipedia.org/all-access/all-agents/"+str(pagetitle)+"/daily/"+from_date+"/"+to_date
    data = requests.get(url=url, headers=headers).json()
    list_views = extractViews(data)
    return list_views

def getViewsRedirects(list_redirects, event_date, days, language):
    from_date = datetime.strptime(event_date,'%Y-%m-%dT%H:%M:%SZ')
    time_range = dateutil.relativedelta.relativedelta(days=days)
    to_date = from_date + time_range
    
    from_date = from_date.strftime('%Y%m%d')
    to_date = to_date.strftime('%Y%m%d')
    
    list_views = []
    for title in list_redirects:
        url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/"+language+".wikipedia.org/all-access/all-agents/"+str(title)+"/daily/"+from_date+"/"+to_date
        data = requests.get(url=url, headers=headers).json()
        views = extractViews(data)
        list_views = list_views + views
    return list_views

def getViewsBefore(pagetitle, page_creation, event_date, days, language):
    to_date = datetime.strptime(event_date,'%Y-%m-%dT%H:%M:%SZ')
    from_date = datetime.strptime(page_creation,'%Y-%m-%dT%H:%M:%SZ')
    
    if(from_date == to_date):
        return []
    
    to_date = to_date - dateutil.relativedelta.relativedelta(days=1)
    time_range = dateutil.relativedelta.relativedelta(days=days)
    from_date = to_date - time_range
    
    from_date = from_date.strftime('%Y%m%d')
    to_date = to_date.strftime('%Y%m%d')
    
    url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/"+language+".wikipedia.org/all-access/all-agents/"+str(pagetitle)+"/daily/"+from_date+"/"+to_date
    data = requests.get(url=url, headers=headers).json()
    list_views = extractViews(data)
    return list_views

def aggregateViews(list_views,func):
    if len(list_views) > 0:
        return func(list_views)
    else:
        return 0

def increase(list_views):
    first = list_views[0]
    last = list_views[-1]
    return (last-first)

In [12]:
days = 6
# get pageviews store them in list
df['list_views_7_days'] = df.apply(lambda row : getViews(row['pagetitle'],row['event_date'],days,language), axis = 1)
df['list_views_7_days_redirects'] = df.apply(lambda row : getViewsRedirects(row['redirects'],row['event_date'],days,language), axis = 1)
df['list_views_before'] = df.apply(lambda row : getViewsBefore(row['pagetitle'],row['page_creation'],row['event_date'],days,language), axis = 1)

In [ ]:
#aggregate pageview lists
df['views_before_mean'] = df.apply(lambda row : aggregateViews(row['list_views_before'],np.mean), axis = 1)
df['views_before_sum'] = df.apply(lambda row : aggregateViews(row['list_views_before'],sum), axis = 1)
df['views_before_max'] = df.apply(lambda row : aggregateViews(row['list_views_before'],max), axis = 1)
df['views_before_min'] = df.apply(lambda row : aggregateViews(row['list_views_before'],min), axis = 1)
df['views_before_median'] = df.apply(lambda row : aggregateViews(row['list_views_before'],np.median), axis = 1)
df['views_before_increase'] = df.apply(lambda row : aggregateViews(row['list_views_before'],increase), axis = 1)

df['views_7_days_sum'] = df.apply(lambda row : aggregateViews(row['list_views_7_days'],sum), axis = 1)
df['views_7_days_mean'] = df.apply(lambda row : aggregateViews(row['list_views_7_days'],np.mean), axis = 1)
df['views_7_redirects_sum'] = df.apply(lambda row : aggregateViews(row['list_views_7_days_redirects'],sum), axis = 1)

df["views_7_sum"] = df['views_7_days_sum'] + df['views_7_redirects_sum']

In [ ]:
##################################################################################################################

In [73]:
countries = pd.read_csv("countries.csv", delimiter = ',')
def getRegion(country):
    countries = pd.read_csv("countries.csv", delimiter = ',')
    if countries["name"].isin([country]).any():
        row = countries[countries["name"]==country]
        return row.iloc[0]["economic_region"]
    
def getContinent(country):
    countries = pd.read_csv("countries.csv", delimiter = ',')
    if countries["name"].isin([country]).any():
        row = countries[countries["name"]==country]
        return row.iloc[0]["maxmind_continent"]

In [74]:
df["country"].replace({"United States of America": "United States", "People's Republic of China": "China"}, inplace=True)
df["country"].replace({"Hamburg": "Germany"}, inplace=True)
df["country"].replace({"Republic of Artsakh": "Azerbaijan"}, inplace=True)
df["country"].replace({"Turkish Republic of Northern Cyprus": "Cyprus"}, inplace=True)
df["country"].replace({"Chechen Republic": "Russia"}, inplace=True)
df["country"].replace({"The Bahamas": "Bahamas"}, inplace=True)
df["country"].replace({"The Gambia": "Gambia"}, inplace=True)
df["country"].replace({"Danish Realm": "Faroe Islands"}, inplace=True)
df["country"].replace({"North Macedonia": "Macedonia"}, inplace=True)
df["country"].replace({"Kingdom of the Netherlands": "Netherlands"}, inplace=True)
df["country"].replace({"Republic of Ireland": "Ireland"}, inplace=True)

In [75]:
df['economic_region'] = df.apply(lambda row : getRegion(row['country']), axis = 1)
df['continent'] = df.apply(lambda row : getContinent(row['country']), axis = 1)

In [ ]:
print("south: ", df[df['economic_region'] == "Global South"]["country"].count())
print("north: ", df[df['economic_region'] == "Global North"]["country"].count())
print("not assigned: ", df[df['economic_region'].isnull() == True]["country"].count())
df[df['economic_region'].isnull() == True]

In [77]:
df = df[df['economic_region'].isnull() == False]

In [ ]:
##################################################################################################################

In [5]:
def getCategories(pagetitle, language):
    URL = "https://"+language+".wikipedia.org/w/api.php"
    PARAMS = {
        "action": "query",
        "format": "json",
        "prop": "categories",
        "titles": pagetitle,
        "cllimit":"100"
    }
    data = requests.get(url=URL, params=PARAMS).json()
    page = data["query"]["pages"]
    categories = []
    try:
        for key, value in page.items():
            for cat in value["categories"]:
                categories.append(cat["title"])
    except:
        categories = []
    return categories

In [6]:
#DE:
def filterCategory(categories,pagetitle):
    category = []
    if ("preisverleihung" in pagetitle.lower()):
        category.append("culture")
    if ("award" in pagetitle.lower()):
        category.append("culture")
    if ("musik" in pagetitle.lower()):
        category.append("culture")
    if ("film" in pagetitle.lower()):
        category.append("culture")
    if ("kriminalfall" in pagetitle.lower()):
        category.append("disaster")
    if ("anschlag" in pagetitle.lower()):
        category.append("disaster")
    if ("unfall" in pagetitle.lower()):
        category.append("disaster")
    if ("katastrophe" in pagetitle.lower()):
        category.append("disaster")
    if ("erdbeben" in pagetitle.lower()):
        category.append("disaster")
    if ("konflikt" in pagetitle.lower()):
        category.append("disaster")
    if ("polit" in pagetitle.lower()):
        category.append("politics")
    if ("wahl" in pagetitle.lower()):
        category.append("politics")
    if ("parlament" in pagetitle.lower()):
        category.append("politics")
    if ("fußball" in pagetitle.lower()):
        category.append("sports")
    if ("sport" in pagetitle.lower()):
        category.append("sports")
    if ("rennen" in pagetitle.lower()):
        category.append("sports")
    if ("turnier" in pagetitle.lower()):
        category.append("sports")
    if ("marathon" in pagetitle.lower()):
        category.append("sports")
    if ("olymp" in pagetitle.lower()):
        category.append("sports")
    if ("polit" in pagetitle.lower()):
        category.append("sports")
    if ("biathlon" in pagetitle.lower()):
        category.append("sports")
    if ("championship" in pagetitle.lower()):
        category.append("sports")
    if ("darts" in pagetitle.lower()):
        category.append("sports")
    if ("badminton" in pagetitle.lower()):
        category.append("sports")
    if ("open" in pagetitle.lower()):
        category.append("sports")
    if ("football" in pagetitle.lower()):
        category.append("sports")
    if("rugby" in pagetitle.lower()):
        category.append("sports")
    if("tour" in pagetitle.lower()):
        category.append("sports")
    if("volleyball" in pagetitle.lower()):
        category.append("sports")
    if categories:
        for cat in categories:
            if ("preisverleihung" in cat.lower()):
                category.append("culture")
            if ("award" in cat.lower()):
                category.append("culture")
            if ("musik" in cat.lower()):
                category.append("culture")
            if ("film" in cat.lower()):
                category.append("culture")
            if ("kriminalfall" in cat.lower()):
                category.append("disaster")
            if ("anschlag" in cat.lower()):
                category.append("disaster")
            if ("unfall" in cat.lower()):
                category.append("disaster")
            if ("katastrophe" in cat.lower()):
                category.append("disaster")
            if ("erdbeben" in cat.lower()):
                category.append("disaster")
            if ("konflikt" in cat.lower()):
                category.append("disaster")
            if ("polit" in cat.lower()):
                category.append("politics")
            if ("wahl" in cat.lower()):
                category.append("politics")
            if ("parlament" in cat.lower()):
                category.append("politics")
            if ("fußball" in cat.lower()):
                category.append("sports")
            if ("sport" in cat.lower()):
                category.append("sports")
            if ("rennen" in cat.lower()):
                category.append("sports")
            if ("turnier" in cat.lower()):
                category.append("sports")
            if ("marathon" in cat.lower()):
                category.append("sports")
            if ("olymp" in cat.lower()):
                category.append("sports")
            if ("rallye" in cat.lower()):
                category.append("sports")
            if ("biathlon" in cat.lower()):
                category.append("sports")
            if ("championship" in cat.lower()):
                category.append("sports")
            if ("darts" in cat.lower()):
                category.append("sports")
            if ("badminton" in cat.lower()):
                category.append("sports")
            if ("open" in cat.lower()):
                category.append("sports")
            if ("football" in cat.lower()):
                category.append("sports")
            if("rugby" in cat.lower()):
                category.append("sports")
            if("tour" in cat.lower()):
                category.append("sports")
            if("volleyball" in cat.lower()):
                category.append("sports")
    most_common = Counter(category).most_common(1)
    if len(most_common) > 0:
        return most_common[0][0]
    else:
        #print(pagetitle,"\n")
        #print(categories)
        #print("\n")
        return "NA"

In [79]:
#EN:
def filterCategory(categories,pagetitle):
    category = []
    if ("pageant" in pagetitle.lower()):
        category.append("culture")
    if ("award" in pagetitle.lower()):
        category.append("culture")
    if ("music" in pagetitle.lower()):
        category.append("culture")
    if ("film" in pagetitle.lower()):
        category.append("culture")
    if ("song contest" in pagetitle.lower()):
        category.append("culture")
    if ("fire" in pagetitle.lower()):
        category.append("disaster")
    if ("massacres" in pagetitle.lower()):
        category.append("disaster")
    if ("bombing" in pagetitle.lower()):
        category.append("disaster")
    if ("flood" in pagetitle.lower()):
        category.append("disaster")
    if ("kill" in pagetitle.lower()):
        category.append("disaster")
    if ("murder" in pagetitle.lower()):
        category.append("disaster")
    if ("crime" in pagetitle.lower()):
        category.append("disaster")
    if ("attack" in pagetitle.lower()):
        category.append("disaster")
    if ("accident" in pagetitle.lower()):
        category.append("disaster")
    if ("incident" in pagetitle.lower()):
        category.append("disaster")
    if ("disaster" in pagetitle.lower()):
        category.append("disaster")
    if ("earthquake" in pagetitle.lower()):
        category.append("disaster")
    if ("landslide" in pagetitle.lower()):
        category.append("disaster")
    if ("conflict" in pagetitle.lower()):
        category.append("disaster")
    if ("cyclone" in pagetitle.lower()):
        category.append("disaster")
    if ("missing person" in pagetitle.lower()):
        category.append("disaster")
    if ("politi" in pagetitle.lower()):
        category.append("politics")
    if ("election" in pagetitle.lower()):
        category.append("politics")
    if ("parliament" in pagetitle.lower()):
        category.append("politics")
    if ("referendum" in pagetitle.lower()):
        category.append("politics")
    if ("football" in pagetitle.lower()):
        category.append("sports")
    if ("sport" in pagetitle.lower()):
        category.append("sports")
    if ("competition" in pagetitle.lower()):
        category.append("sports")
    if ("olymp" in pagetitle.lower()):
        category.append("sports")
    if ("summit" in pagetitle.lower()):
        category.append("politics")
    if ("government" in pagetitle.lower()):
        category.append("politics")
    if ("championship" in pagetitle.lower()):
        category.append("sports")
    if ("badminton" in pagetitle.lower()):
        category.append("sports")
    if ("open" in pagetitle.lower()):
        category.append("sports")
    if ("football" in pagetitle.lower()):
        category.append("sports")
    if("rugby" in pagetitle.lower()):
        category.append("sports")
    if("tour" in pagetitle.lower()):
        category.append("sports")
    if("cup" in pagetitle.lower()):
        category.append("sports")
    if("wrestling" in pagetitle.lower()):
        category.append("sports")
    if ("basketball" in pagetitle.lower()):
        category.append("sports")
    if ("winter games" in pagetitle.lower()):
        category.append("sports")
    if ("soccer" in pagetitle.lower()):
        category.append("sports")
    if ("biathlon" in pagetitle.lower()):
        category.append("sports")
    if ("boxing" in pagetitle.lower()):
        category.append("sports")
    if ("skating" in pagetitle.lower()):
        category.append("sports")
    if ("winter universiade" in pagetitle.lower()):
        category.append("sports")
    if categories:
        for cat in categories:
            if ("pageant" in cat.lower()):
                category.append("culture")
            if ("award" in cat.lower()):
                category.append("culture")
            if ("music" in cat.lower()):
                category.append("culture")
            if ("film" in cat.lower()):
                category.append("culture")
            if ("song contest" in cat.lower()):
                category.append("culture")
            if ("fire" in cat.lower()):
                category.append("disaster")
            if ("massacres" in cat.lower()):
                category.append("disaster")
            if ("bombing" in cat.lower()):
                category.append("disaster")
            if ("flood" in cat.lower()):
                category.append("disaster")
            if ("kill" in cat.lower()):
                category.append("disaster")
            if ("murder" in cat.lower()):
                category.append("disaster")
            if ("crime" in cat.lower()):
                category.append("disaster")
            if ("attack" in cat.lower()):
                category.append("disaster")
            if ("accident" in cat.lower()):
                category.append("disaster")
            if ("incident" in cat.lower()):
                category.append("disaster")
            if ("disaster" in cat.lower()):
                category.append("disaster")
            if ("earthquake" in cat.lower()):
                category.append("disaster")
            if ("landslide" in cat.lower()):
                category.append("disaster")
            if ("conflict" in cat.lower()):
                category.append("disaster")
            if ("cyclone" in cat.lower()):
                category.append("disaster")
            if ("missing person" in cat.lower()):
                category.append("disaster")
            if ("summit" in cat.lower()):
                category.append("politics")
            if ("government" in cat.lower()):
                category.append("politics")
            if ("politi" in cat.lower()):
                category.append("politics")
            if ("election" in cat.lower()):
                category.append("politics")
            if ("parliament" in cat.lower()):
                category.append("politics")
            if ("referendum" in cat.lower()):
                category.append("politics")
            if ("football" in cat.lower()):
                category.append("sports")
            if ("sport" in cat.lower()):
                category.append("sports")
            if ("competition" in cat.lower()):
                category.append("sports")
            if ("olymp" in cat.lower()):
                category.append("sports")
            if ("rallye" in cat.lower()):
                category.append("sports")
            if ("championship" in cat.lower()):
                category.append("sports")
            if ("badminton" in cat.lower()):
                category.append("sports")
            if ("open" in cat.lower()):
                category.append("sports")
            if ("cup" in cat.lower()):
                category.append("sports")
            if("rugby" in cat.lower()):
                category.append("sports")
            if("tour" in cat.lower()):
                category.append("sports")
            if("volleyball" in cat.lower()):
                category.append("sports")
            if("wrestling" in cat.lower()):
                category.append("sports")
            if ("basketball" in cat.lower()):
                category.append("sports")
            if ("winter games" in cat.lower()):
                category.append("sports")
            if ("soccer" in cat.lower()):
                category.append("sports")
            if ("biathlon" in cat.lower()):
                category.append("sports")
            if ("boxing" in cat.lower()):
                category.append("sports")
            if ("skating" in cat.lower()):
                category.append("sports")
            if ("winter universiade" in cat.lower()):
                category.append("sports")
    most_common = Counter(category).most_common(1)
    if len(most_common) > 0:
        return most_common[0][0]
    else:
        #print(pagetitle,"\n")
        #print(categories)
        #print("\n")
        return ""
    

In [ ]:
#IT:
def filterCategory(categories,pagetitle):
    category = []
    if ("cinema" in pagetitle.lower()):
        category.append("culture")
    if ("award" in pagetitle.lower()):
        category.append("culture")
    if ("music" in pagetitle.lower()):
        category.append("culture")
    if ("film" in pagetitle.lower()):
        category.append("culture")
    if ("televisione" in pagetitle.lower()):
        category.append("culture")
    if ("miss " in pagetitle.lower()):
        category.append("culture")
    if ("omicidio" in pagetitle.lower()):
        category.append("disaster")
    if ("catastrof" in pagetitle.lower()):
        category.append("disaster")
    if ("terroris" in pagetitle.lower()):
        category.append("disaster")
    if ("terremot" in pagetitle.lower()):
        category.append("disaster")
    if ("attentat" in pagetitle.lower()):
        category.append("disaster")
    if ("disastr" in pagetitle.lower()):
        category.append("disaster")
    if ("incident" in pagetitle.lower()):
        category.append("disaster")
    if ("massacr" in pagetitle.lower()):
        category.append("disaster")
    if ("strage" in pagetitle.lower()):
        category.append("disaster")
    if ("morte" in pagetitle.lower()):
        category.append("disaster")
    if ("attacc" in pagetitle.lower()):
        category.append("disaster")
    if ("guerra" in pagetitle.lower()):
        category.append("disaster")
    if ("climate" in pagetitle.lower()):
        category.append("politics")
    if ("politic" in pagetitle.lower()):
        category.append("politics")
    if ("elezion" in pagetitle.lower()):
        category.append("politics")
    if ("parlamentari" in pagetitle.lower()):
        category.append("politics")
    if ("referendum" in pagetitle.lower()):
        category.append("politics")
    if ("presidenziali" in pagetitle.lower()):
        category.append("politics")
    if ("calcistiche" in pagetitle.lower()):
        category.append("sports")
    if ("ciclismo" in pagetitle.lower()):
        category.append("sports")
    if ("competizion" in pagetitle.lower()):
        category.append("sports")
    if ("giochi" in pagetitle.lower()):
        category.append("sports")
    if ("calcio" in pagetitle.lower()):
        category.append("sports")
    if ("campionat" in pagetitle.lower()):
        category.append("sports")
    if ("ciclistiche" in pagetitle.lower()):
        category.append("sports")
    if ("olimpiade" in pagetitle.lower()):
        category.append("sports")
    if ("pallavolo" in pagetitle.lower()):
        category.append("sports")
    if ("atletica" in pagetitle.lower()):
        category.append("sports")
    if("olimpic" in pagetitle.lower()):
        category.append("sports")
    if("football" in pagetitle.lower()):
        category.append("sports")
    if("wta" in pagetitle.lower()):
        category.append("sports")
    if("ufc" in pagetitle.lower()):
        category.append("sports")
    if ("atp" in pagetitle.lower()):
        category.append("sports")
    if ("coppa" in pagetitle.lower()):
        category.append("sports")
    if ("sport" in pagetitle.lower()):
        category.append("sports")
    if ("pallacanestro" in pagetitle.lower()):
        category.append("sports")
    if ("motociclismo" in pagetitle.lower()):
        category.append("sports")
    if ("corsa" in pagetitle.lower()):
        category.append("sports")
    if ("champion" in pagetitle.lower()):
        category.append("sports")
    if ("gran premio" in pagetitle.lower()):
        category.append("sports")
    if ("wrestling" in pagetitle.lower()):
        category.append("sports")
    if ("open " in pagetitle.lower()):
        category.append("sports")
    if ("cup" in pagetitle.lower()):
        category.append("sports")
    if ("baseball" in pagetitle.lower()):
        category.append("sports")
    if categories:
        for cat in categories:
            if ("cinema" in cat.lower()):
                category.append("culture")
            if ("award" in cat.lower()):
                category.append("culture")
            if ("music" in cat.lower()):
                category.append("culture")
            if ("film" in cat.lower()):
                category.append("culture")
            if ("televisione" in cat.lower()):
                category.append("culture")
            if ("miss " in cat.lower()):
                category.append("culture")
            if ("omicidio" in cat.lower()):
                category.append("disaster")
            if ("catastrof" in cat.lower()):
                category.append("disaster")
            if ("terroris" in cat.lower()):
                category.append("disaster")
            if ("terremot" in cat.lower()):
                category.append("disaster")
            if ("attentat" in cat.lower()):
                category.append("disaster")
            if ("disastr" in cat.lower()):
                category.append("disaster")
            if ("incident" in cat.lower()):
                category.append("disaster")
            if ("massacr" in cat.lower()):
                category.append("disaster")
            if ("strage" in cat.lower()):
                category.append("disaster")
            if ("morte" in cat.lower()):
                category.append("disaster")
            if ("attacc" in cat.lower()):
                category.append("disaster")
            if ("guerra" in cat.lower()):
                category.append("disaster")
            if ("climate" in cat.lower()):
                category.append("politics")
            if ("politic" in cat.lower()):
                category.append("politics")
            if ("elezion" in cat.lower()):
                category.append("politics")
            if ("parlamentari" in cat.lower()):
                category.append("politics")
            if ("referendum" in cat.lower()):
                category.append("politics")
            if ("presidenziali" in cat.lower()):
                category.append("politics")
            if ("calcistiche" in cat.lower()):
                category.append("sports")
            if ("ciclismo" in cat.lower()):
                category.append("sports")
            if ("competizion" in cat.lower()):
                category.append("sports")
            if ("giochi" in cat.lower()):
                category.append("sports")
            if ("calcio" in cat.lower()):
                category.append("sports")
            if ("campionat" in cat.lower()):
                category.append("sports")
            if ("ciclistiche" in cat.lower()):
                category.append("sports")
            if ("olimpiade" in cat.lower()):
                category.append("sports")
            if ("pallavolo" in cat.lower()):
                category.append("sports")
            if ("atletica" in cat.lower()):
                category.append("sports")
            if("olimpic" in cat.lower()):
                category.append("sports")
            if("football" in cat.lower()):
                category.append("sports")
            if("wta" in cat.lower()):
                category.append("sports")
            if("ufc" in cat.lower()):
                category.append("sports")
            if ("atp" in cat.lower()):
                category.append("sports")
            if ("coppa" in cat.lower()):
                category.append("sports")
            if ("sport" in cat.lower()):
                category.append("sports")
            if ("pallacanestro" in cat.lower()):
                category.append("sports")
            if ("motociclismo" in cat.lower()):
                category.append("sports")
            if ("corsa" in cat.lower()):
                category.append("sports")
            if ("champion" in cat.lower()):
                category.append("sports")
            if ("gran premio" in cat.lower()):
                category.append("sports")
            if ("wrestling" in cat.lower()):
                category.append("sports")
            if ("open " in cat.lower()):
                category.append("sports")
            if ("cup" in cat.lower()):
                category.append("sports")
            if ("baseball" in cat.lower()):
                category.append("sports")
    most_common = Counter(category).most_common(1)
    if len(most_common) > 0:
        return most_common[0][0]
    else:
        #print(pagetitle,"\n")
        #print(categories)
        #print("\n")
        return ""


In [258]:
#ES:
def filterCategory(categories,pagetitle):
    category = []
    if ("cine" in pagetitle.lower()):
        category.append("culture")
    if ("award" in pagetitle.lower()):
        category.append("culture")
    if ("belleza" in pagetitle.lower()):
        category.append("culture")
    if ("música" in pagetitle.lower()):
        category.append("culture")
    if ("artístico" in pagetitle.lower()):
        category.append("culture")
    if ("miss " in pagetitle.lower()):
        category.append("culture")
    if ("televisión" in pagetitle.lower()):
        category.append("culture")
    if ("premios" in pagetitle.lower()):
        category.append("culture")
    if ("atentados" in pagetitle.lower()):
        category.append("disaster")
    if ("terrorist" in pagetitle.lower()):
        category.append("disaster")
    if ("terrorismo" in pagetitle.lower()):
        category.append("disaster")
    if ("accident" in pagetitle.lower()):
        category.append("disaster")
    if ("incident" in pagetitle.lower()):
        category.append("disaster")
    if ("desastre" in pagetitle.lower()):
        category.append("disaster")
    if ("terremoto" in pagetitle.lower()):
        category.append("disaster")
    if ("masacre" in pagetitle.lower()):
        category.append("disaster")
    if ("pandemia" in pagetitle.lower()):
        category.append("disaster")
    if ("asesinato" in pagetitle.lower()):
        category.append("disaster")
    if ("eleccion" in pagetitle.lower()):
        category.append("politics")
    if ("incendio" in pagetitle.lower()):
        category.append("disaster")
    if ("guerra" in pagetitle.lower()):
        category.append("disaster")
    if ("polític" in pagetitle.lower()):
        category.append("politics")
    if ("parlament" in pagetitle.lower()):
        category.append("politics")
    if ("referendum" in pagetitle.lower()):
        category.append("politics")
    if ("presidenciale" in pagetitle.lower()):
        category.append("politics")
    if ("referéndum" in pagetitle.lower()):
        category.append("politics")
    if ("deport" in pagetitle.lower()):
        category.append("sports")
    if ("fútbol" in pagetitle.lower()):
        category.append("sports")
    if ("ciclismo" in pagetitle.lower()):
        category.append("sports")
    if ("competicion" in pagetitle.lower()):
        category.append("sports")
    if ("olímpi" in pagetitle.lower()):
        category.append("sports")
    if ("ciclistiche" in pagetitle.lower()):
        category.append("sports")
    if ("olimpiade" in pagetitle.lower()):
        category.append("sports")
    if ("torneo" in pagetitle.lower()):
        category.append("sports")
    if ("campeonat" in pagetitle.lower()):
        category.append("sports")
    if("motociclismo" in pagetitle.lower()):
        category.append("sports")
    if ("voleibol" in pagetitle.lower()):
        category.append("sports")
    if ("rugby" in pagetitle.lower()):
        category.append("sports")
    if ("patinaje" in pagetitle.lower()):
        category.append("sports")
    if ("atletismo" in pagetitle.lower()):
        category.append("sports")
    if ("concursos" in pagetitle.lower()):
        category.append("sports")
    if ("baloncesto" in pagetitle.lower()):
        category.append("sports")
    if ("champion" in pagetitle.lower()):
        category.append("sports")
    if ("tenis" in pagetitle.lower()):
        category.append("sports")
    if ("wrestling" in pagetitle.lower()):
        category.append("sports")
    if ("open " in pagetitle.lower()):
        category.append("sports")
    if ("cup " in pagetitle.lower()):
        category.append("sports")
    if ("baseball" in pagetitle.lower()):
        category.append("sports")
    if ("copa" in pagetitle.lower()):
        category.append("sports")
    if ("uefa" in pagetitle.lower()):
        category.append("sports")
    if ("league" in pagetitle.lower()):
        category.append("sports")
    if ("challenger" in pagetitle.lower()):
        category.append("sports")
    if ("tour de francia" in pagetitle.lower()):
        category.append("sports")
    if ("giro de italia" in pagetitle.lower()):
        category.append("sports")
    if ("la vuelta a españa" in pagetitle.lower()):
        category.append("sports")
    if categories:
        for cat in categories:
            if ("cine" in cat.lower()):
                category.append("culture")
            if ("award" in cat.lower()):
                category.append("culture")
            if ("belleza" in cat.lower()):
                category.append("culture")
            if ("música" in cat.lower()):
                category.append("culture")
            if ("artístico" in cat.lower()):
                category.append("culture")
            if ("miss " in cat.lower()):
                category.append("culture")
            if ("televisión" in cat.lower()):
                category.append("culture")
            if ("premios" in cat.lower()):
                category.append("culture")
            if ("atentados" in cat.lower()):
                category.append("disaster")
            if ("terrorist" in cat.lower()):
                category.append("disaster")
            if ("terrorismo" in cat.lower()):
                category.append("disaster")
            if ("accident" in cat.lower()):
                category.append("disaster")
            if ("incident" in cat.lower()):
                category.append("disaster")
            if ("desastre" in cat.lower()):
                category.append("disaster")
            if ("terremoto" in cat.lower()):
                category.append("disaster")
            if ("masacre" in cat.lower()):
                category.append("disaster")
            if ("pandemia" in cat.lower()):
                category.append("disaster")
            if ("asesinato" in cat.lower()):
                category.append("disaster")
            if ("incendio" in cat.lower()):
                category.append("disaster")
            if ("eleccion" in cat.lower()):
                category.append("politics")
            if ("guerra" in cat.lower()):
                category.append("disaster")
            if ("polític" in cat.lower()):
                category.append("politics")
            if ("parlament" in cat.lower()):
                category.append("politics")
            if ("referendum" in cat.lower()):
                category.append("politics")
            if ("presidenciale" in cat.lower()):
                category.append("politics")
            if ("referéndum" in cat.lower()):
                category.append("politics")
            if ("deport" in cat.lower()):
                category.append("sports")
            if ("fútbol" in cat.lower()):
                category.append("sports")
            if ("ciclismo" in cat.lower()):
                category.append("sports")
            if ("competicion" in cat.lower()):
                category.append("sports")
            if ("olímpi" in cat.lower()):
                category.append("sports")
            if ("ciclistiche" in cat.lower()):
                category.append("sports")
            if ("olimpiade" in cat.lower()):
                category.append("sports")
            if ("torneo" in cat.lower()):
                category.append("sports")
            if ("campeonat" in cat.lower()):
                category.append("sports")
            if("motociclismo" in cat.lower()):
                category.append("sports")
            if ("voleibol" in cat.lower()):
                category.append("sports")
            if ("rugby" in cat.lower()):
                category.append("sports")
            if ("patinaje" in cat.lower()):
                category.append("sports")
            if ("atletismo" in cat.lower()):
                category.append("sports")
            if ("concursos" in cat.lower()):
                category.append("sports")
            if ("baloncesto" in cat.lower()):
                category.append("sports")
            if ("champion" in cat.lower()):
                category.append("sports")
            if ("tenis" in cat.lower()):
                category.append("sports")
            if ("wrestling" in cat.lower()):
                category.append("sports")
            if ("open " in cat.lower()):
                category.append("sports")
            if ("cup " in cat.lower()):
                category.append("sports")
            if ("baseball" in cat.lower()):
                category.append("sports")
            if ("uefa" in cat.lower()):
                category.append("sports")
            if ("league" in cat.lower()):
                category.append("sports")
            if ("challenger" in cat.lower()):
                category.append("sports")
            if ("tour de francia" in cat.lower()):
                category.append("sports")
            if ("giro de italia" in cat.lower()):
                category.append("sports")
            if ("la vuelta a españa" in cat.lower()):
                category.append("sports")
    most_common = Counter(category).most_common(1)
    if len(most_common) > 0:
        return most_common[0][0]
    else:
        #print(pagetitle,"\n")
        #print(categories)
        #print("\n")
        return ""

In [17]:
df['categories'] = df.apply(lambda row : getCategories(row['pagetitle'],language), axis = 1)

In [170]:
df['cat'] = df.apply(lambda row : filterCategory(row['categories'],row['pagetitle']), axis = 1)

Rowing at the 2015 Pan American Games 

['Category:Articles with short description', 'Category:Rowing at the 2015 Pan American Games', 'Category:Short description is different from Wikidata']


2015 Copa do Nordeste 

['Category:All articles lacking sources', 'Category:Articles lacking sources from April 2014', 'Category:Articles with short description', 'Category:Copa do Nordeste', 'Category:Short description matches Wikidata']


Volleyball at the 2015 Southeast Asian Games – Women 

['Category:Redirects connected to a Wikidata item', 'Category:Redirects from merges', 'Category:Redirects to sections']


Beach soccer at the 2015 Mediterranean Beach Games 

['Category:2015 in beach soccer', 'Category:Articles with short description', 'Category:Beach soccer at the Mediterranean Beach Games', 'Category:CS1 Italian-language sources (it)', 'Category:Short description matches Wikidata', 'Category:Use dmy dates from November 2019']


2015 National Women's Soccer League season 

["Category:201

In [210]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

cat_list = np.concatenate( df.categories, axis=0 )

list_tokens = []
for i in range(len(cat_list)):
    tokens = word_tokenize(cat_list[i])
    list_tokens.append(tokens)
list_tokens = np.concatenate(list_tokens, axis=0)

stopwords = nltk.corpus.stopwords.words('italian')
newStopWords = ['category','articles','description','short']
stopwords.extend(newStopWords)
filtered_sentence = []

for w in list_tokens:
    if w not in stopwords:
        filtered_sentence.append(w)
        
filtered_sentence = [word.lower() for word in filtered_sentence]
filtered_sentence = [word for word in filtered_sentence if word.isalpha()]

word_frequ = FreqDist(filtered_sentence)

In [280]:
#word_frequ
#word_frequ.most_common(200)

In [ ]:
print("politics: ", df[df['cat'] == "politics"]["cat"].count())
print("sports: ", df[df['cat'] == "sports"]["cat"].count())
print("culture: ", df[df['cat'] == "culture"]["cat"].count())
print("disaster: ", df[df['cat'] == "disaster"]["cat"].count())
#print("not assigned: ", df[df['cat'] == ""]["cat"].count())

In [67]:
def extractEdits(json_data):
    list_edits = []
    if "items" in json_data:
        if "results" in json_data["items"][0]:
            for item in json_data["items"][0]["results"]:
                list_edits.append(item["edits"])
    return list_edits
            

def getEditsBefore(pagetitle,  page_creation, event_date, days, language):
    to_date = datetime.strptime(event_date,'%Y-%m-%dT%H:%M:%SZ')
    from_date = datetime.strptime(page_creation,'%Y-%m-%dT%H:%M:%SZ')
    
    if(from_date == to_date):
        return []
    
    to_date = to_date - dateutil.relativedelta.relativedelta(days=1)
    time_range = dateutil.relativedelta.relativedelta(days=days)
    from_date = to_date - time_range
    
    from_date = from_date.strftime('%Y%m%d')
    to_date = to_date.strftime('%Y%m%d')
    
    url = "https://wikimedia.org/api/rest_v1/metrics/edits/per-page/"+language+".wikipedia/"+str(pagetitle)+"/all-editor-types/daily/"+from_date+"/"+to_date
    data = requests.get(url=url, headers=headers).json()
    list_edits = extractEdits(data)
    return list_edits

def getEdits(pagetitle, event_date, start_date, days, language):
    from_date = datetime.strptime(event_date,'%Y-%m-%dT%H:%M:%SZ')      
    time_range = dateutil.relativedelta.relativedelta(days=days)
    to_date = from_date + time_range
    
    from_date = from_date.strftime('%Y%m%d')
    to_date = to_date.strftime('%Y%m%d')
    url = "https://wikimedia.org/api/rest_v1/metrics/edits/per-page/"+language+".wikipedia/"+str(pagetitle)+"/all-editor-types/daily/"+from_date+"/"+to_date
    data = requests.get(url=url, headers=headers).json()
    list_edits = extractEdits(data)
    return list_edits

def getEditsRedirects(list_redirects, event_date, days, language):
    from_date = datetime.strptime(event_date,'%Y-%m-%dT%H:%M:%SZ')
    time_range = dateutil.relativedelta.relativedelta(days=days)
    to_date = from_date + time_range
    
    from_date = from_date.strftime('%Y%m%d')
    to_date = to_date.strftime('%Y%m%d')
    
    list_edits = []
    for title in list_redirects:
        url = "https://wikimedia.org/api/rest_v1/metrics/edits/per-page/"+language+".wikipedia/"+str(title)+"/all-editor-types/daily/"+from_date+"/"+to_date
        data = requests.get(url=url, headers=headers).json()
        edits = extractEdits(data)
        list_edits = list_edits + edits
    return list_edits

In [68]:
df['list_edits_redirects'] = df.apply(lambda row : getEditsRedirects(row['redirects'],row['event_date'],6,language), axis = 1)
df

,event_url,event_label,date,country,start_date,end_date,event_date,event_id,pagetitle,redirects,...,list_views_7_days_redirects,diff_days,list_views_before,categories,economic_region,continent,cat,list_edits_before,list_edits,list_edits_redirects
0,http://www.wikidata.org/entity/Q20647547,2015 Kazakhstan First Division,2015-01-01T00:00:00Z,Kazakhstan,2015-04-10T00:00:00Z,2015-11-14T00:00:00Z,2015-04-10T00:00:00Z,Q20647547,2015 Kazakhstan First Division,[],...,[],86,[],[Category:2014–15 in European second tier asso...,Global South,Asia,sports,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]",[]
1,http://www.wikidata.org/entity/Q20647776,2015 Greek football scandal,2015-01-01T00:00:00Z,Greece,2015-04-06T00:00:00Z,NaN,2015-04-06T00:00:00Z,Q20647776,2015 Greek football match-fixing scandal,"[2015 Greek football scandal, 2015 Greek footb...",...,[],35,[],"[Category:2014–15 in Greek football, Category:...",Global North,Europe,sports,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,http://www.wikidata.org/entity/Q20647949,2015 Washington wildfires,2015-01-01T00:00:00Z,United States,2015-06-28T00:00:00Z,NaN,2015-06-28T00:00:00Z,Q20647949,2015 Washington wildfires,"[2015 washington state wildfires, 2015 Washing...",...,"[130, 72, 61, 40]",1,[],"[Category:2015 in Washington (state), Category...",Global North,North America,disaster,"[0, 0, 0, 0, 0, 0]","[0, 13, 2, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,http://www.wikidata.org/entity/Q20648362,2015 Reinert Open,2015-01-01T00:00:00Z,Germany,2015-07-06T00:00:00Z,2015-07-12T00:00:00Z,2015-07-06T00:00:00Z,Q20648362,2015 Reinert Open,[],...,[],-6,"[9, 33, 23, 20, 43]","[Category:2015 ITF Women's Circuit, Category:2...",Global North,Europe,sports,"[0, 1, 0, 1, 0, 0]","[1, 1, 0, 0, 0, 4]",[]
4,http://www.wikidata.org/entity/Q18348667,2015 Cork Senior Hurling Championship,2015-01-01T00:00:00Z,Ireland,2015-04-11T00:00:00Z,NaN,2015-04-11T00:00:00Z,Q18348667,2015 Cork Senior Hurling Championship,[],...,[],-181,[],[Category:2015 senior hurling county champions...,Global North,Europe,sports,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 5, 0, 0]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10564,http://www.wikidata.org/entity/Q105616790,Results of the 2020 New Hampshire House of Rep...,2020-11-03T00:00:00Z,United States,NaN,NaN,2020-11-03T00:00:00Z,Q105616790,Results of the 2020 New Hampshire House of Rep...,[],...,[],95,[],"[Category:2020 New Hampshire elections, Catego...",Global North,North America,politics,"[0, 0, 0, 0, 0, 0]",[],[]
10565,http://www.wikidata.org/entity/Q100791760,protests against the tightening of the abortio...,2020-01-01T00:00:00Z,Poland,2020-10-22T00:00:00Z,http://www.wikidata.org/.well-known/genid/d805...,2020-10-22T00:00:00Z,Q100791760,2020–2021 women's strike protests in Poland,"[2020 Polish abortion protests, October 2020 P...",...,[5406],5,[],"[Category:2020 protests, Category:Abortion-rig...",Global North,Europe,NaN,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 21]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10566,http://www.wikidata.org/entity/Q85711929,2019–20 Confederation presidential primary,2020-01-18T00:00:00Z,United States,NaN,NaN,2020-01-18T00:00:00Z,Q85711929,2019–20 Confederation presidential primary,"[2020 Confederation presidential primary, 2019...",...,"[93, 36, 31, 28, 15, 18, 18]",-80,"[241, 209, 144, 87, 75, 96, 80]","[Category:2019 elections in Poland, Category:2...",Global North,North America,politics,"[4, 3, 3, 0, 1, 1]",[],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10567,http://www.wikidata.org/entity/Q85735165,2020 Civic Democratic Party leadership election,2020-01-18T00:00:00Z,Czech Republic,NaN,NaN,2020-01-18T00:00:00Z,Q85735165,2020 Civic Democratic Party leadership election,[],...,[],-54,"[8, 4, 4, 4, 14, 4, 13]",[Category:2020 elections in the Czech Republic...,Global North,Europe,politics,"[0, 0, 0, 0, 0, 0]","[3, 0, 0, 0, 4, 0]",[]


In [43]:
df['list_edits_before'] = df.apply(lambda row : getEditsBefore(row['pagetitle'],row['page_creation'],row['event_date'],6,language), axis = 1)
df.tail()

,event_url,event_label,date,country,start_date,end_date,event_date,event_id,pagetitle,redirects,page_creation,list_views_7_days,list_views_7_days_redirects,diff_days,list_views_before,categories,economic_region,continent,cat,list_edits_before
10564,http://www.wikidata.org/entity/Q105616790,Results of the 2020 New Hampshire House of Rep...,2020-11-03T00:00:00Z,United States,NaN,NaN,2020-11-03T00:00:00Z,Q105616790,Results of the 2020 New Hampshire House of Rep...,[],2021-02-06T23:33:39Z,[],[],95,[],"[Category:2020 New Hampshire elections, Catego...",Global North,North America,politics,"[0, 0, 0, 0, 0, 0]"
10565,http://www.wikidata.org/entity/Q100791760,protests against the tightening of the abortio...,2020-01-01T00:00:00Z,Poland,2020-10-22T00:00:00Z,http://www.wikidata.org/.well-known/genid/d805...,2020-10-22T00:00:00Z,Q100791760,2020–2021 women's strike protests in Poland,"[2020 Polish abortion protests, October 2020 P...",2020-10-27T05:31:49Z,[],[5406],5,[],"[Category:2020 protests, Category:Abortion-rig...",Global North,Europe,NaN,"[0, 0, 0, 0, 0, 0]"
10566,http://www.wikidata.org/entity/Q85711929,2019–20 Confederation presidential primary,2020-01-18T00:00:00Z,United States,NaN,NaN,2020-01-18T00:00:00Z,Q85711929,2019–20 Confederation presidential primary,"[2020 Confederation presidential primary, 2019...",2019-10-30T23:58:57Z,"[361, 111, 90, 75, 45, 40, 22]","[93, 36, 31, 28, 15, 18, 18]",-80,"[241, 209, 144, 87, 75, 96, 80]","[Category:2019 elections in Poland, Category:2...",Global North,North America,politics,"[4, 3, 3, 0, 1, 1]"
10567,http://www.wikidata.org/entity/Q85735165,2020 Civic Democratic Party leadership election,2020-01-18T00:00:00Z,Czech Republic,NaN,NaN,2020-01-18T00:00:00Z,Q85735165,2020 Civic Democratic Party leadership election,[],2019-11-25T16:02:00Z,"[67, 8, 3, 3, 37, 2, 1]",[],-54,"[8, 4, 4, 4, 14, 4, 13]",[Category:2020 elections in the Czech Republic...,Global North,Europe,politics,"[0, 0, 0, 0, 0, 0]"
10568,http://www.wikidata.org/entity/Q85735761,2020 Green Party (Czech Republic) leadership e...,2020-01-25T00:00:00Z,Czech Republic,NaN,NaN,2020-01-25T00:00:00Z,Q85735761,2020 Green Party (Czech Republic) leadership e...,[],2020-01-18T18:35:18Z,"[70, 5, 9, 7, 1, 5, 7]",[],-7,"[20, 9, 3, 5, 8, 4, 4]",[Category:2020 elections in the Czech Republic...,Global North,Europe,politics,"[1, 0, 0, 0, 0, 0]"


In [44]:
df['list_edits'] = df.apply(lambda row : getEdits(row['pagetitle'],row['event_date'],row['start_date'],6,language), axis = 1)
df.tail()

,event_url,event_label,date,country,start_date,end_date,event_date,event_id,pagetitle,redirects,...,list_views_7_days,list_views_7_days_redirects,diff_days,list_views_before,categories,economic_region,continent,cat,list_edits_before,list_edits
10564,http://www.wikidata.org/entity/Q105616790,Results of the 2020 New Hampshire House of Rep...,2020-11-03T00:00:00Z,United States,NaN,NaN,2020-11-03T00:00:00Z,Q105616790,Results of the 2020 New Hampshire House of Rep...,[],...,[],[],95,[],"[Category:2020 New Hampshire elections, Catego...",Global North,North America,politics,"[0, 0, 0, 0, 0, 0]",[]
10565,http://www.wikidata.org/entity/Q100791760,protests against the tightening of the abortio...,2020-01-01T00:00:00Z,Poland,2020-10-22T00:00:00Z,http://www.wikidata.org/.well-known/genid/d805...,2020-10-22T00:00:00Z,Q100791760,2020–2021 women's strike protests in Poland,"[2020 Polish abortion protests, October 2020 P...",...,[],[5406],5,[],"[Category:2020 protests, Category:Abortion-rig...",Global North,Europe,NaN,"[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 21]"
10566,http://www.wikidata.org/entity/Q85711929,2019–20 Confederation presidential primary,2020-01-18T00:00:00Z,United States,NaN,NaN,2020-01-18T00:00:00Z,Q85711929,2019–20 Confederation presidential primary,"[2020 Confederation presidential primary, 2019...",...,"[361, 111, 90, 75, 45, 40, 22]","[93, 36, 31, 28, 15, 18, 18]",-80,"[241, 209, 144, 87, 75, 96, 80]","[Category:2019 elections in Poland, Category:2...",Global North,North America,politics,"[4, 3, 3, 0, 1, 1]",[]
10567,http://www.wikidata.org/entity/Q85735165,2020 Civic Democratic Party leadership election,2020-01-18T00:00:00Z,Czech Republic,NaN,NaN,2020-01-18T00:00:00Z,Q85735165,2020 Civic Democratic Party leadership election,[],...,"[67, 8, 3, 3, 37, 2, 1]",[],-54,"[8, 4, 4, 4, 14, 4, 13]",[Category:2020 elections in the Czech Republic...,Global North,Europe,politics,"[0, 0, 0, 0, 0, 0]","[3, 0, 0, 0, 4, 0]"
10568,http://www.wikidata.org/entity/Q85735761,2020 Green Party (Czech Republic) leadership e...,2020-01-25T00:00:00Z,Czech Republic,NaN,NaN,2020-01-25T00:00:00Z,Q85735761,2020 Green Party (Czech Republic) leadership e...,[],...,"[70, 5, 9, 7, 1, 5, 7]",[],-7,"[20, 9, 3, 5, 8, 4, 4]",[Category:2020 elections in the Czech Republic...,Global North,Europe,politics,"[1, 0, 0, 0, 0, 0]",[]


In [338]:
##################################################################################################################

In [10]:
def getPlanedValue(diff_days):
    if(diff_days < 0):
        return "planed"
    else:
        return "not_planed"
    
def getSurprisingValue(diff_days):
    if(diff_days == 0 | diff_days == 1):
        return "surprising"
    else:
        return "not_surprising"
    
def getThreeFactorValue(diff_days):
    if(diff_days < 0):
        return "before"
    if(diff_days == 0 | diff_days == 1):
        return "surprising"
    if(diff_days > 1):
        return "after"

In [11]:
df['year'] = pd.DatetimeIndex(df['event_date']).year
df['surprising'] = df.apply(lambda row : getSurprisingValue(row['diff_days']), axis = 1)
df['planed'] = df.apply(lambda row : getPlanedValue(row['diff_days']), axis = 1)
df['factor'] = df.apply(lambda row: getThreeFactorValue(row['diff_days']), axis = 1)

In [6]:
#df.event_id

In [13]:
#aggregate pageview lists
df['views_before_mean'] = df.apply(lambda row : aggregateViews(row['list_views_before'],np.mean), axis = 1)
df['views_before_sum'] = df.apply(lambda row : aggregateViews(row['list_views_before'],sum), axis = 1)
df['views_before_max'] = df.apply(lambda row : aggregateViews(row['list_views_before'],max), axis = 1)
df['views_before_min'] = df.apply(lambda row : aggregateViews(row['list_views_before'],min), axis = 1)
df['views_before_median'] = df.apply(lambda row : aggregateViews(row['list_views_before'],np.median), axis = 1)
df['views_before_increase'] = df.apply(lambda row : aggregateViews(row['list_views_before'],increase), axis = 1)

df['views_7_days_sum'] = df.apply(lambda row : aggregateViews(row['list_views_7_days'],sum), axis = 1)
df['views_7_days_mean'] = df.apply(lambda row : aggregateViews(row['list_views_7_days'],np.mean), axis = 1)
df['views_7_redirects_sum'] = df.apply(lambda row : aggregateViews(row['list_views_7_days_redirects'],sum), axis = 1)

df["views_7_sum"] = df['views_7_days_sum'] + df['views_7_redirects_sum']

In [14]:
df['edits_before_mean'] = df.apply(lambda row : aggregateViews(row['list_edits_before'],np.mean), axis = 1)
df['edits_before_sum'] = df.apply(lambda row : aggregateViews(row['list_edits_before'],sum), axis = 1)
df['edits_before_max'] = df.apply(lambda row : aggregateViews(row['list_edits_before'],max), axis = 1)
df['edits_before_min'] = df.apply(lambda row : aggregateViews(row['list_edits_before'],min), axis = 1)
df['edits_before_median'] = df.apply(lambda row : aggregateViews(row['list_edits_before'],np.median), axis = 1)
df['edits_before_increase'] = df.apply(lambda row : aggregateViews(row['list_edits_before'],increase), axis = 1)

df['edits_7_days_sum'] = df.apply(lambda row : aggregateViews(row['list_edits'],sum), axis = 1)
df['edits_7_days_mean'] = df.apply(lambda row : aggregateViews(row['list_edits'],np.mean), axis = 1)
df['edits_7_redirects_sum'] = df.apply(lambda row : aggregateViews(row['list_edits_redirects'],sum), axis = 1)

df["edits_7_sum"] = df['edits_7_days_sum'] + df['edits_7_redirects_sum']

In [15]:
# get values for analysis
df = df[['event_id','pagetitle','event_date','economic_region','country','continent','cat','categories','views_7_sum','views_before_mean','views_before_sum','views_before_max','views_before_min','views_before_median','views_before_increase','list_views_7_days','diff_days','year','planed','surprising','factor','page_creation','edits_7_sum','edits_before_min','edits_before_mean','edits_before_max','edits_before_median','edits_before_sum']]
df.head()

,event_id,pagetitle,event_date,economic_region,country,continent,cat,categories,views_7_sum,views_before_mean,...,planed,surprising,factor,page_creation,edits_7_sum,edits_before_min,edits_before_mean,edits_before_max,edits_before_median,edits_before_sum
0,Q18413459,US Open 2015,2015-08-31T00:00:00Z,Global North,United States,North America,sports,[Kategorie:US Open 2015],17059,630.142857,...,planed,not_surprising,before,2014-10-07T16:23:12Z,3,0,0.333333,1,0.0,2
1,Q18450378,Kurzbahneuropameisterschaften 2015,2015-12-02T00:00:00Z,Global North,Israel,Asia,sports,"[Kategorie:Netanja, Kategorie:Schwimmeuropamei...",2038,16.428571,...,planed,not_surprising,before,2015-11-23T12:07:35Z,56,0,0.000000,0,0.0,0
2,Q18577969,Tippeligaen 2015,2015-04-06T00:00:00Z,Global North,Norway,Europe,sports,"[Kategorie:Eliteserien-Saison (Fußball), Kateg...",0,0.000000,...,planed,not_surprising,before,2015-02-28T13:04:10Z,6,0,0.000000,0,0.0,0
3,Q18589121,BMW Malaysian Open 2015,2015-03-02T00:00:00Z,Global South,Malaysia,Asia,sports,"[Kategorie:Tennisturnier in Kuala Lumpur, Kate...",0,0.000000,...,planed,not_surprising,before,2015-02-26T09:07:42Z,37,0,0.166667,1,0.0,1
4,Q18589152,Abierto Monterrey Afirme 2015,2015-03-01T00:00:00Z,Global South,Mexico,North America,sports,"[Kategorie:Tennisturnier in Monterrey, Kategor...",0,0.000000,...,planed,not_surprising,before,2015-02-26T09:12:44Z,24,0,0.166667,1,0.0,1


In [16]:
# pageview are only available from 1.7.2015
df = df.sort_values(by="event_date").reset_index(drop=True)
df["event_date"] = pd.to_datetime(df["event_date"], format='%Y-%m-%dT%H:%M:%SZ')
# delete everything before july 2015
df = df[df.event_date > datetime.strptime("2015-06-30T00:00:00Z",'%Y-%m-%dT%H:%M:%SZ')].reset_index(drop=True)
df = df[df.event_date < datetime.strptime("2021-01-01T00:00:00Z",'%Y-%m-%dT%H:%M:%SZ')].reset_index(drop=True)
df["pagetitle"].count()

4547

In [17]:
df.to_csv (r'events_dataframe_de.csv', index = False, header=True)